<a href="https://colab.research.google.com/github/ben0it8/poetry-language-model/blob/master/arany_language_model_wordpiece.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Install extra requirements

In [0]:
!pip install -U ftfy
!pip install -U sentencepiece

    100% |████████████████████████████████| 51kB 2.0MB/s 
    100% |████████████████████████████████| 1.0MB 19.0MB/s 


In [0]:
import os, sys
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
%matplotlib inline
%config InlineBackend.figure_format='retina'; # adapt plots for retina displays
import seaborn as sns
sns.set_style('darkgrid');
sns.set_context(context='notebook');
from typing import List
from tqdm import tqdm_notebook as tqdm
from pathlib import Path
from pprint import pprint
import numpy as np 
import math
import time
import pandas as pd
import io
import ftfy 
import re
import requests
import os
import torch
from collections import defaultdict
import dill
import pickle

import torch
from torch import nn, optim, tensor
from torch.nn import functional as F
from random import sample

from google.colab import files
from zipfile import ZipFile

import sentencepiece as spm

logger = logging.getLogger(__name__)

data_dir = Path("data/arany").resolve()
data_dir.mkdir(exist_ok=True, parents=True)
url = "https://raw.githubusercontent.com/ben0it8/ady/master/data/arany.txt" 

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [0]:
def get_file_paths(path):
  path = Path(path)
  ret = []
  for x in path.glob('*'):
    if x.is_file() and not str(x).endswith('.zip'):
      ret.append(str(x))
  return ret

def zip_dir(path, name):
  if not name.endswith('.zip'): name += '.zip'
  file_paths = get_file_paths(path)
  
  with ZipFile(path/name, 'w') as zip:
    for file in file_paths:
      zip.write(file, os.path.basename(file))
  print(f"Zipped files at {path}")
  return (path/name).resolve()

##Read data

## Clean data & write to disk

In [0]:
def is_title(t):
  t = re.sub('[\d\s\s\-!?,]', '', t)
  return t.isupper()

def fix_text(t:str):
  t = ftfy.fix_text(t, normalization='NFKC')
  t = t.replace('\n', '') # remove newlines
  t = re.sub(r'[»«]', '', t) # remove special parenthesis
  t = re.sub(r'[0-9]','', t)
  t = re.sub("\s\s+", " ", t) # skip whitespaces
  t = re.sub("A.J.", "", t)
  t = re.sub("A. J.", "", t)
  t = t.strip()
  return t
    
def fix_texts(texts:list):
  out = []
  for i, line in enumerate(texts):
    if "\u2424" in line:
      line = line.split("\u2424")
    elif '\u2028' in line:
      line = line.split('\u2028')
    elif '\u000A' in line:
      line = line.split('\u000A')
    else:
      line = [line] 
    for t in line:
      t = fix_text(t)
      if (t is None or len(t.replace(' ', ''))<=3 or is_title(t) or
          t.startswith(('.', ',', '?', '!', '-', ';')) or len(t) > 100):
        continue
      else:
        out += [t]   
  return out

response =  requests.get(url)
texts = [line.decode() for line in response.iter_lines()]
print(f"No. of lines: {len(texts)}")
clean_texts = fix_texts(texts)
(data_dir/'text_clean.txt').open(mode='wt').writelines(f"{line}\n" for line in clean_texts)
print(f"No. of lines: {len(clean_texts)}")
np.random.shuffle(clean_texts)

No. of lines: 57384
No. of lines: 49495


## Define Tokenizer (which uses SentencePiece), Corpus ( data handler)

In [0]:
class Tokenizer():
    
    ID_UNK = 0
    ID_SOS = 1
    ID_EOS = 2
    
    def __init__(self, model_path:str):
        self.sp = spm.SentencePieceProcessor()
        self.sp.Load(str(model_path))
        self.id_unk = self.sp.unk_id()
        self.tk_unk = self.sp.IdToPiece(self.id_unk)
        self.tk_sos = self.sp.IdToPiece(self.ID_SOS)
        self.tk_eos = self.sp.IdToPiece(self.ID_EOS)
        self.vocab_size = len(self.sp)
        logger.info(f"Initialized SentPieceProcessor from {model_path}")
    
    def numericalize(self, tokens: List[str]) -> List[List[int]]:
        if isinstance(tokens, str): tokens = [tokens]
        ids =  [self.sp.EncodeAsIds(s) for s in tokens]
        if len(ids) == 1: ids=ids[0]
        return ids

    def piecify(self, tokens: List[str]) -> List[List[str]]:
        if isinstance(tokens, str): tokens = [tokens]
        return [self.sp.EncodeAsPieces(s) for s in tokens]
    
    def textify(self, ids: List[int]) -> str:
        if isinstance(ids, list) and isinstance(ids[0], np.generic): 
            ids = [int(x) for x in ids]
        if not isinstance(ids, list) and not isinstance(ids[0], int):
            raise TypeError("Argument `ids` has to be a list of integers.")            
        return self.sp.DecodeIds(ids)
    
    @classmethod
    def from_file(cls, input_file:str, output_path:str='default', vocab_size:int=16000, 
                  char_cov:float=1.0, model_type:str='unigram'):
        
        assert model_type in ['unigram', 'bpe', 'char', 'word']
        assert 0 < char_cov <= 1
        input_file = str(input_file)
        output_file =  os.path.splitext(str(output_path))[0]
        ext = '.model'
        train_cmd = f"--input={input_file} --model_prefix={output_file}"\
                    f" --vocab_size={vocab_size} --character_coverage={char_cov} --model_type={model_type}"

        logger.info(f"Train command: {train_cmd}")
        logger.info(f"Started training SentencePiece model...")
        ret = spm.SentencePieceTrainer.Train(train_cmd)
        logger.info(f"Exit code: {int(ret)}")
        return cls(output_file+ext)
      
def batchify(data, bsz):
    # work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

class Corpus(object):

  def __init__(self, file_path, tokenizer, bs=20):
       
        self.processor = tokenizer
        self.id_sos, self.id_eos = self.processor.ID_SOS, self.processor.ID_EOS
        self.bs = bs
        self.data = self.tokenize(file_path)        
        self.vocab_size = self.processor.vocab_size
        
  def tokenize(self, path):
      """Tokenizes a text file."""
      ids = []
      with open(path, 'r') as f:
          for line in f:
              numericalized = self.processor.numericalize(line)
              ids.extend([self.id_sos] + numericalized + [self.id_eos])

      return batchify(torch.LongTensor(ids), self.bs)

## Define RNN model architecture,  training loop and helpers

In [0]:
class RNNModel(nn.Module):

    def __init__(self, rnn_type, ntoken, emsize, nhid, nlayers, dropout=0.5, tie_weights=False):
        super(RNNModel, self).__init__()
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, emsize)
        assert (rnn_type in ['LSTM', 'GRU']), "Arg `rnn_type` has to be one of {GRU, LSTM}."
        self.rnn = getattr(nn, rnn_type)(emsize, nhid, nlayers, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)

        if tie_weights:
            if nhid != emsize:
                raise ValueError('When using the tied flag, nhid must be equal to emsize')
            self.decoder.weight = self.encoder.weight

        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, input, hidden):
        emb = self.drop(self.encoder(input))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters())
        if self.rnn_type == 'LSTM':
            return (weight.new_zeros(self.nlayers, bsz, self.nhid),
                    weight.new_zeros(self.nlayers, bsz, self.nhid))
        else:
            return weight.new_zeros(self.nlayers, bsz, self.nhid)
       
      
def repackage_hidden(h):
  """Wraps hidden states in new Tensors, to detach them from their history."""
  if isinstance(h, torch.Tensor):
    return h.detach()
  else:
    return tuple(repackage_hidden(v) for v in h)
      
def get_num_params(model):
  return sum([p.nelement() for p in model.parameters() if p.requires_grad])

def get_batch(source, i, bptt):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target
  
def train_epoch(train_data, model, vocab_size, bs=16, bptt=20, clip=.25):
    model.train()
    total_loss = 0.0
    hidden = model.init_hidden(bs)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i, bptt)

        # Starting each batch, we detach the hidden state from how it was previously produced.
        # If we didn't, the model would try backpropagating all the way to start of the dataset.
        hidden = repackage_hidden(hidden)
        model.zero_grad()
        output, hidden = model(data, hidden)
        loss = criterion(output.view(-1, vocab_size), targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        total_loss += len(data) * loss.item()
        
    return total_loss / (len(train_data) - 1)

In [0]:
# INIT AND TRAIN TOKENIZER FROM CLEAN TEXTS
bs = 64

model_type = 'unigram' # can be "bpe"/"unigram" to support wordpieces

vocab_size = 20000 # how many wordpieces to consider

tokenizer = Tokenizer.from_file(data_dir/'text_clean.txt', 
                                output_path=data_dir/'tokenizer', char_cov=1.0,
                                model_type=model_type, vocab_size=vocab_size) 

# INIT CORPUS FROM CLEAN TEXTS AND TOKENIZER WITH BATCH_SIZE `BS`

corpus = Corpus(data_dir/'text_clean.txt', tokenizer=tokenizer, bs=bs)

2019-04-10 13:50:13,550 : INFO : Train command: --input=/content/data/arany/text_clean.txt --model_prefix=/content/data/arany/tokenizer --vocab_size=20000 --character_coverage=1.0 --model_type=unigram
2019-04-10 13:50:13,552 : INFO : Started training SentencePiece model...
2019-04-10 13:50:22,044 : INFO : Exit code: 1
2019-04-10 13:50:22,085 : INFO : Initialized SentPieceProcessor from /content/data/arany/tokenizer.model


In [0]:
# LSTM PARAMETERS
model_type='GRU' # less params than LSTM

emsize = 400
nhid = 400

nlayers = 1

dropout = 0.1 # > 0.2 gave hard time converging
clip = 3.0 # lower seemmed to make convergence difficult

tied = True # ideal for word/wordpiece level

bptt = 80 # the higher the more context info; compute trade-off

lr = 1e-3

# INIT LSTM MODEL, LOSS FUNCTION AND OPTIMIZER

model = RNNModel(model_type, corpus.vocab_size, emsize, nhid, nlayers, dropout, tied).to(device)
print(f"No. of parameters: {get_num_params(model)}")

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:46: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


No. of parameters: 8982400


## Let's train the model

In [0]:
epochs = 150

start_time = time.time()
try:
  
  for epoch in range(1, epochs+1):

    loss = train_epoch(corpus.data, model, corpus.vocab_size, 
                       clip=clip, bs=bs, bptt=bptt)
    elapsed = time.time() - start_time
    print("epoch {:3d} | loss {:5.2f} | perplexity {:8.2f}| elapsed {:5.2f}s ".format(epoch, loss, math.exp(loss), elapsed))

except KeyboardInterrupt:
  print('-' * 89)
  print('Exiting from training early')

finally:
  params = {"model_type": model_type,
            "ntoken": corpus.vocab_size,
            "emsize": emsize,
            "nhid": nhid,
            "nlayers": nlayers,
            "dropout": dropout,
            "tied": tied}
 
  with open(data_dir/'model_state.pth', 'wb') as f:
    torch.save({"state_dict": model.state_dict(),
                "params": params}, f)
    
zipfile = zip_dir(data_dir, data_dir.name)

epoch   1 | loss  6.60 | perplexity   737.11| elapsed 20.99s 
epoch   2 | loss  5.87 | perplexity   355.47| elapsed 41.88s 
epoch   3 | loss  5.65 | perplexity   285.71| elapsed 62.77s 
epoch   4 | loss  5.48 | perplexity   239.01| elapsed 83.69s 
epoch   5 | loss  5.32 | perplexity   204.84| elapsed 104.66s 
epoch   6 | loss  5.18 | perplexity   177.63| elapsed 125.68s 
epoch   7 | loss  5.04 | perplexity   153.96| elapsed 146.68s 
epoch   8 | loss  4.89 | perplexity   133.44| elapsed 167.64s 
epoch   9 | loss  4.75 | perplexity   115.37| elapsed 188.59s 
epoch  10 | loss  4.61 | perplexity   100.31| elapsed 209.62s 
epoch  11 | loss  4.47 | perplexity    87.46| elapsed 230.65s 
epoch  12 | loss  4.33 | perplexity    76.19| elapsed 251.71s 
epoch  13 | loss  4.20 | perplexity    66.61| elapsed 272.68s 
epoch  14 | loss  4.07 | perplexity    58.63| elapsed 293.68s 
epoch  15 | loss  3.95 | perplexity    51.78| elapsed 314.71s 
epoch  16 | loss  3.83 | perplexity    45.97| elapsed 335.7

## Generate verses! 

In [0]:
from random import sample
import torch

def is_unbalanced(s):
  if s.count('"') % 2 != 0 or s.count('(') != s.count(')'):
    return True
  else:
    return False

def sample_punkt():
  return sample(['.', '?', '!'], 1)[0]

def parse_last_line(s):
  l = list(s)
  if l[-1] == ',': 
    l[-1] = sample_punkt()
  
  if l[-1] not in list('.?!'): 
    l.append(sample_punkt())  
  return "".join(l)
  
def generate_line(model, hidden=None, temp=1.0, 
               sos_id=1, eos_id=2, unk_id=0, max_len=None):
  """Generate line from `model` with `hidden` state at `temp`."""
  ids = []
  
  if hidden is None:
    hidden = model.init_hidden(1)
  
  input = torch.tensor([sos_id], dtype=torch.long).reshape(1,1).to(device)
  
  id = 0
  while id != eos_id and len(ids)<max_len :
    output, hidden = model(input, hidden)
    probs = output.squeeze().div(temp).exp().cpu() 
    id = torch.multinomial(probs, num_samples=1).item() 
    if id == sos_id or id == unk_id: continue
    input.fill_(id)
    ids += [id]
  
  return ids, hidden

def generate(model, tokenizer, num_lines=8, min_len=8, max_len=15,
             unk_id=0, sos_id=1, eos_id=2, temp=0.6):
  """ 
  Generate a verse consisting of `num_lines` lines of max. length `max_tokens`.
  Since the hidden state is passed onto the next line, 
  observing some cross-line consistency would be expected, or less
  optimistically, at least grammatically correct sentences.
  NOTE: line length can be tuned by changing max_tokens (i.e. subword pieces).
  
  Args:
    model: Trained PyTorch language model
    tokenizer: SentencePiece tokenizer
    temp: Temperature parameter; lower: more conservative, higher: more diverse
    num_lines: No. of lines to generate.
    max_len: Max no. of tokens per line (not words!)
    sos_id: Start of sequence id in vocabulary
    eos_id: End of sequence id in vocabulary
  
  Returns: list of strings
  """
  model.eval()
  
  lines = []
  line_cnt = 0
  hidden = model.init_hidden(1)

  with torch.no_grad():
    
    while line_cnt != num_lines:
      try:

        ids, hidden = generate_line(model, hidden=hidden, temp=temp, max_len=max_len,
                                    sos_id=sos_id, eos_id=eos_id, unk_id=unk_id)
        
        if len(ids) <= min_len: raise Exception
        line = tokenizer.textify(ids).strip()
        
        if line.startswith(tuple("-?!.,()")): raise Exception
        if is_unbalanced(line): raise Exception
        
        lines += [line]
        line_cnt +=1
        
      except Exception as e:
        pass
    
  last_line = lines.pop()
  l = parse_last_line(last_line)
  lines.append(l)
  
  return lines

In [0]:
generate(model, tokenizer, temp=0.5, num_lines=10, min_len=8, max_len=10)

['Elmondá az útat, egészen Budáig:',
 't, ha még most is elcsutak',
 'vén, hogy ezt meg nem adja,',
 'én; ha nem lesz ez, hogy kid',
 'ünk - azaz hogy nem hitte senki;',
 'ünk, ha majd csak egy-néha igaz,',
 'Nosza, Károly járatlan hangi és fűzött',
 'Szellem volt, mely a csatatér semmi szemet',
 'Ezt hallván a táborhez félre,',
 'ét a bokronk Doronghynipjára?']

In [0]:
generate(model, tokenizer, temp=0.6, num_lines=10, min_len=8, max_len=10)

['Megrezzent a leány eme hideg kéztől,',
 'Werner, szája, az anyja, az elás',
 'ármányaitól visszatéren vens',
 'Mi az istennyila, féltvén az özvegynek,',
 ': s az apró evóik tan-',
 'en a késő király, a vén Bence.',
 'a nép, ha volna elcseitták',
 'aféle nemzet, ti a tenger nép.',
 'vagy csak nehány a halottak nem úgy, mint egy',
 'a kocsihoz, gerézdét?']

In [0]:
generate(model, tokenizer, temp=0.7, num_lines=10, min_len=8, max_len=10)

["mondok, vón' a nagy szél-tor",
 'Erántadék: nincs, korussa',
 'ad, ha nem is, mi a nemcsak',
 'yuram, Egy szem a magyar ember,',
 'a két nép gőzilesz... vagy -',
 'Úgy szólott ahozról, ki a minapi',
 'adék, s vele lőn a mai párság',
 'a terem eluntig jó minapi.',
 'ig . Odaták be a .',
 'égen, ki a repűlést használtatott.']